In [ ]:
!pip install pymupdf

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"  # Extract text from each page
    return text

# Example Usage
pdf_file = "Resume.pdf"
text = extract_text_from_pdf(pdf_file)
print(text)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 54.4 MB/s eta 0:00:00


FileNotFoundError: no such file: 'Resume.pdf'

In [ ]:
import nltk
nltk.download('stopwords')
import string
import pickle
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained model and vectorizer
model = pickle.load(open('model.pkl', 'rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))

# Preprocess text (cleaning function)
def preprocess_text(text):
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words("english"))  # Load stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text

# Function to split large text into smaller chunks
def split_text(text, chunk_size=50):
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Function to detect plagiarism for long texts
def detect_large_text(input_text, similarity_threshold=0.7):
    chunks = split_text(input_text)  # Split large text into smaller chunks
    processed_chunks = [preprocess_text(chunk) for chunk in chunks]  # Preprocess chunks
    vectorized_chunks = tfidf_vectorizer.transform(processed_chunks)  # Convert to TF-IDF

    predictions = model.predict(vectorized_chunks)  # Get model predictions

    # Cosine similarity check
    similarity_scores = cosine_similarity(vectorized_chunks, vectorized_chunks)
    max_similarity = np.max(similarity_scores)

    # If any chunk is plagiarized OR similarity is high, mark as plagiarism
    if 1 in predictions or max_similarity > similarity_threshold:
        return "Plagiarism Detected"
    return "No Plagiarism"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(detect_large_text(text))

Plagiarism Detected


In [ ]:
import nltk
nltk.download('stopwords')
import string
import pickle
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))

# Preprocess text (cleaning function)
def preprocess_text(text):
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words("english"))  # Load stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text

# Function to detect plagiarism for large texts
def detect_large_text(input_text, similarity_threshold=0.7):
    processed_text = preprocess_text(input_text)  # Preprocess entire text
    vectorized_text = tfidf_vectorizer.transform([processed_text])  # Convert to TF-IDF

    # Get model predictions
    prediction = model.predict(vectorized_text)

    # Cosine similarity check (assuming you compare with a database)
    similarity_scores = cosine_similarity(vectorized_text, vectorized_text)
    max_similarity = np.max(similarity_scores)

    # If model detects plagiarism OR similarity is high, mark as plagiarism
    if prediction[0] == 1 or max_similarity > similarity_threshold:
        return "Plagiarism Detected"
    return "No Plagiarism"




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Example usage
input_text = """Machine learning teaches computers to recognize patterns and make decisions automatically using data and algorithms. It can be broadly categorized into three types:
Supervised Learning: Trains models on labeled data to predict or classify new, unseen data.
Unsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.
Reinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks."""
result = detect_large_text(input_text)
print(result)

Plagiarism Detected


In [ ]:
input_text = """I am an engineering student love to code."""
result = detect_large_text(input_text)
print(result)

Plagiarism Detected
